### General rules:
 * For all figures that you generate, remember to add meaningful labels to the axes, and make a legend, if applicable.
 * Do not hard code constants, like number of samples, number of channels, etc in your program. These values should always be determined from the given data. This way, you can easily use the code to analyse other data sets.
 * Do not use high-level functions from toolboxes like scikit-learn.
 * Replace *Template* by your *FirstnameLastname* in the filename, or by *Lastname1Lastname2* if you work in pairs.

# BCI-IL WS 2018/2019 - Exercise Sheet #04

#### Name:

In [1]:
% matplotlib inline

import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

import bci_minitoolbox as bci
import bci_classifiers as cfy

## Exercise 1: Implementation of Shrinkage for Covariance Estimation  (7 points)
Implement a function `cov_shrink` that estimates the covariance matrix of data using shrinkage with the analytic method of determining the shrinkage coefficient as presented in the lecture. Input and output of that function should be as in the function `numpy.cov`.

If you cannot succeed with this task, you may import the function `cov_shrink` from `bci_helper_sheet4.pyc` (available at the moodle page) for the subsequent exercises.

In [2]:
def cov_shrink(X):
    '''
    Estimate covariance of given data using shrinkage estimator.
    
    Synopsis:
        C= cov_shrink(X)
    Argument:
        X: data matrix (features x samples)
    Output:
        C: estimated covariance matrix
    '''
    for i in range(X.shape[1]):
        avg = np.mean(X[:, i])
        Z[i] = (X[:,i] - avg).dot((X[:, i] - avg).T)
    sigma_cap = np.sum(Z)/Z.shape[0]-1
    #Added mean, we forgot that last time from the formula
    v = np.mean(np.trace(sigma_cap)/sigam_cap.shape[0])
    
    s = sigma_cap - v*np.identity(sigma_cap.shape[0])
   
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            var += np.var(Z[:,i,j])
            sum_s += np.square(s[i,j])
        
    gamma_star = (X.shape[1]/np.square((X.shape[1]-1)))*var/sum_s
    
    sigma_tilde = (1-gamma_star)*sigma_cap+gamma_star*v*np.identity
    
    return sigma_tilde
                  
    
    

## Exercise 2: Implementation of LDA with Shrinkage  (3 points)
Implement a function `train_LDAshrink` that calculates the LDA classifier in which the estimation of the covariance matrices is enhanced by shrinkage. Input and output should be the same as for `train_LDA` from sheet \#03. As for LDA, use the pseudo inverse (`numpy.linalg.pinv`) instead of the usual matrix inversion. 

If you cannot succeed with this task, you may import the function `train_LDAshrink` from `bci_helper_sheet04_pythonPV.pyc` (available at the moodle page) with PV being your python version (27,35,36) for the subsequent exercises.

In [3]:
def train_LDAshrink(X, y):
    '''
    Synopsis:
        w, b= train_LDAshrink(X, y)
    Arguments:
        X: data matrix (features X samples)
        y: labels with values 0 and 1 (1 x samples)
    Output:
        w: LDA weight vector
        b: bias term
    '''
    
    u1 = np.mean(X[:, y == 0], axis=1)
    u2 = np.mean(X[:, y == 1], axis=1)
    
    #From np.cov documentation:  each row represents a variable, 
    #with observations in the columns
    C_target = cov_shrink(X[:, y == 0])
    C_nontarget = cov_shrink(X[:, y == 1])
    C = 1/2 * (C_target + C_nontarget) 
    w = np.linalg.pinv(C).dot(u2 - u1)
    b = w.T.dot((u1 + u2)/2)
    
    return (w, b)

## Preparation: Load data

In [9]:
fname = 'erp_hexVPsag.npz'
cnt, fs, clab, mnt, mrk_pos, mrk_class, mrk_className = bci.load_data(fname)

## Exercise 3: Classification of Spatio-Temporal Features  (5 points)
First, use the time-averaged potential in each of the intervals [ms]: 160-200, 200-220, 230-260, 260-290, 300-320, 330-370, 380-430, and 480-520 as feature vector (dimension 55 `x` 1) for each trial. For each interval calculate the 3-fold cross-validation error using LDA with and without shrinkage.

In a second step, concatenate the eight feature vectors, that were calcualted for the eight specified intervals into one feature vector (dimension 440 `x` 1) for each trial. Again, determine the 3-fold cross-validation error using LDA with and without shrinkage.

In [73]:
ival = [[160, 200], [200, 220], [230, 260], [260, 290], [300, 320], [330, 370], [380, 430], [480, 520]]

ref_ival = [-100, 0]
epo, epo_t = bci.makeepochs(cnt, fs, mrk_pos, ival[i])
epo = bci.baseline(epo, epo_t, ref_ival)

epo shape: (5, 55, 1200)
tmp shape: (5, 55)
epo shape: (3, 55, 1200)
tmp shape: (3, 55)
epo shape: (4, 55, 1200)
tmp shape: (4, 55)
epo shape: (4, 55, 1200)
tmp shape: (4, 55)
epo shape: (3, 55, 1200)
tmp shape: (3, 55)
epo shape: (5, 55, 1200)
tmp shape: (5, 55)
epo shape: (6, 55, 1200)
tmp shape: (6, 55)
epo shape: (5, 55, 1200)
tmp shape: (5, 55)


/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
